This script generate a single files that contains all gen_solutions result and runtime.

In [ ]:
def get_gen_solution_info(data_root_dir, problem_id):
  json_file_path = data_root_dir+str(problem_id).zfill(4)+'/gen_solutions.json'
  # check if the file exist
  if not os.path.exists(json_file_path):
    return [],[]
  else:
    with open(json_file_path, 'r') as file:
      data = json.load(file)
      runtimes = []
      results = []
      for i in range(0, len(data)):
        runtimes.append(data[i]['time'])
        results.append(data[i]['result'])
      return results, runtimes

# save all data in json file
json_data = {}
for i in range(0, 5000):
    print(i)
    results, runtimes = get_gen_solution_info('data/APPS/train/', i)
    json_data[i] = {}
    json_data[i]['results'] = results
    json_data[i]['runtimes'] = runtimes

with open('models_eval/train_gen_results_runtimes.json', 'w') as file:
    json.dump(json_data, file)

In [ ]:
def get_gen_runtime(data_root_dir, problem_id):
  json_file_path = data_root_dir+str(problem_id).zfill(4)+'/gen_solutions.json'
  # check if the file exist
  if not os.path.exists(json_file_path):
    return []
  else:
    with open(json_file_path, 'r') as file:
      data = json.load(file)
      runtimes = []
      for i in range(0, len(data)):
        if data[i]['time'] != None:
          runtimes.append(data[i]['time'])
      return runtimes

def get_baseline_runtime(data_root_dir, problem_id):
    json_file_path = data_root_dir+str(problem_id).zfill(4)+'/baseline_solutions.json'
    # check if the file exist
    if not os.path.exists(json_file_path):
        return []
    else:
        with open(json_file_path, 'r') as file:
            data = json.load(file)
        if data[0]['time'] == None:
            return []
        return [data[0]['time']]

In [ ]:
data_root_dir = 'data/APPS/train/'
problem_id_baseline_not_solved_gen_solved = []
for i in range(0, 5000):
    gen_runtime = get_gen_runtime(data_root_dir, i)
    baseline_runtime = get_baseline_runtime(data_root_dir, i)
    if len(gen_runtime) != 0 and len(baseline_runtime) == 0:
        problem_id_baseline_not_solved_gen_solved.append(i)
print(len(problem_id_baseline_not_solved_gen_solved))

In [ ]:
import numpy as np
a,b,c,d,e,f = 0,0,0,0,0,0
data_root_dir = 'data/APPS/train/'
count_intro, count_inter, count_comp = 0,0,0
for i in range(0, 5000):
    print(f'Process problem {i}')
    gen_runtime = get_gen_runtime(data_root_dir, i)
    baseline_runtime = get_baseline_runtime(data_root_dir, i)
    if len(gen_runtime) == 0 and len(baseline_runtime) == 0:
      continue
    if len(gen_runtime) !=0 and len(baseline_runtime) !=0:
      gen_avg = np.mean(gen_runtime)
      if gen_avg>baseline_runtime:
        if 0<=i<=1999: c+=1
        elif 2000<=i<=2360: e+=1
        else: a+=1
      else:
        if 0<=i<=1999: d+=1
        elif 2000<=i<=2360: f+=1
        else: b+=1
    if len(baseline_runtime)==0:
        if 0<=i<=1999: count_inter+=1
        elif 2000<=i<=2360: count_comp+=1
        else: count_intro+=1

In [ ]:
print(f'count intro {count_intro}')
print(f'count inter {count_inter}')
print(f'count comp {count_comp}')
print(f'a {a}')
print(f'b {b}')
print(f'c {c}')
print(f'd {d}')
print(f'e {e}')
print(f'f {f}')

In [ ]:
# plot column plot
import matplotlib.pyplot as plt
import numpy as np

# Assuming `data` is your filled data structure
data = {
    'introductory': {
        'gen_higher': a/2639,
        'gen_lower': b/2639,
        'no_bl':count_intro/2639
    },
    'interview': {
        'gen_higher': c/2000,
        'gen_lower': d/2000,
        'no_bl': count_inter/2000
    },
    'competition': {
        'gen_higher': e/361,
        'gen_lower': f/361,
        'no_bl': count_comp/361
    }
}

# Categories
difficulties = list(data.keys())
categories = ['gen_higher', 'gen_lower','no_bl']

# Data for bar heights
gen_higher_values = [data[difficulty]['gen_higher'] for difficulty in difficulties]
gen_lower_values = [data[difficulty]['gen_lower'] for difficulty in difficulties]
no_bl_values = [data[difficulty]['no_bl'] for difficulty in difficulties]

# Position of bars on x-axis
ind = np.arange(len(difficulties))
# Width of a bar
width = 0.2  # Adjusted for three bars

# Plotting
plt.figure(figsize=(12, 7))

plt.bar(ind - width, gen_higher_values, width, label='Avg sampled runtime > baseline runtime')
plt.bar(ind, no_bl_values, width, label='basline solution fails the tests')
plt.bar(ind +width, gen_lower_values, width, label='Avg sampled runtime < baseline runtime')


plt.xlabel('Difficulty Level')
plt.ylabel('% problems')
plt.title('Comparison of Sampled and Baseline Solution Runtimes by Difficulty')

# xticks()
plt.xticks(ind, difficulties)  # Adjusted to align with the center of the groups

# Adding legends
plt.legend(loc='best')
# Show the plot
plt.savefig('models_eval/gen_baseline_runtimes_compare_1.png')
plt.show()

### Process all data

In [ ]:
data_root_dir = 'data/APPS/train/'
# for scatter plot
import matplotlib.pyplot as plt
gen_runtimes = []
baseline_runtimes = []
problem_ids = []
problem_ids_bl = []

for i in range(0, 5000):
    print(f'Process problem {i}')
    gen_runtime = get_gen_runtime(data_root_dir, i)
    baseline_runtime = get_baseline_runtime(data_root_dir, i)
    if len(gen_runtime) == 0 or len(baseline_runtime) == 0:
        continue
    gen_runtimes.extend(gen_runtime)
    baseline_runtimes.extend(baseline_runtime)
    problem_ids.extend([i] * len(gen_runtime))
    problem_ids_bl.append(i)

In [ ]:
import json
data = {}
data['gen_runtimes'] = gen_runtimes
data['baseline_runtimes'] = baseline_runtimes
data['problem_ids'] = problem_ids
data['problem_ids_bl'] = problem_ids_bl
with open('models_eval/base_model_runtime_data.json', 'w') as outfile:
    json.dump(data, outfile)

### Qualatative analysis of the generated programs and runtime

In [ ]:
# To read the results
# load the above json file
with open('models_eval/train_gen_results_runtimes.json', 'r') as file:
    data = json.load(file)
data['38']
# 4174????? 4299

In [ ]:
pass_count = []
pass_3 = []
for i in range(5000):
  result = data[str(i)]['results']
  passed = 0
  for r in result:
    if r == True:
      passed +=1
  pass_count.append(passed)
  if passed ==3:
    pass_3.append(i)

In [ ]:
import matplotlib.pyplot as plt
# Create the histogram
plt.hist(pass_count, bins=range(12), edgecolor='black', align='left')

# Set the labels and title
plt.xlabel('Number of programs that passed all the unit test')
plt.ylabel('Count')
plt.title('Pass@10 analysis for APPS train data')

# Show the plot
plt.show()

In [ ]:
len(good_problems)
good_problems
# Train:
# interview: 0-1999
# competition: 2000-2360
# Intro: 2361-4999

# intro: 2610 2649 2659 2742 2787 2805 3080
# interview: 1439 1606
# comp: 2002 2026 2055 2118 2191 2197

In [ ]:
large_variance_problems = []
for i in good_problems:
  runtime = data[str(i)]['runtimes']
  if runtime and max(runtime)>200:
    large_variance_problems.append(i)
data['2787']

Visualize program

In [ ]:
import json
with open('models_eval/train_gen_results_runtimes.json', 'r') as file:
    loaded_data = json.load(file)

In [ ]:
problem_id = '2169'

In [ ]:
runtimes = loaded_data[problem_id]['runtimes']
file_path = f"./outputs/train/codes/{problem_id}.json"
with open(file_path, 'r') as json_file:
    data = json.load(json_file)
    # Extract the code entries
    code_entries = data[problem_id]["code"]

    # Print each code entry in a readable way
    for idx, code in enumerate(code_entries, 1):
        print(f"Program {idx}:\n")
        print(f'Runtime {runtimes[idx-1]}')
        print(code)
        print("\n" + "-"*50 + "\n")